In [20]:
from __future__ import print_function

# This is important!
# import os
# os.environ["TF_ENABLE_CONTROL_FLOW_V2"] = "1"

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, TimeDistributed, Flatten, Lambda
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import utils
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam

In [21]:
def read_labels(file_path):

    column_names = [
        'experiment ID',
        'user ID',
        'activity ID',
        'start',
        'end'

    ]

    labels_df = pd.read_csv(file_path, delimiter=' ', header=None, names=column_names);

    return labels_df

def read_experiments(labels_path):
    
    exp_path = 'HAPT Data Set/RawData/'
    
    exp_df = read_labels(labels_path)
    acc_data_list_x = []
    acc_data_list_y = []
    acc_data_list_z = []

    gyro_data_list_x = []
    gyro_data_list_y = []
    gyro_data_list_z = []

    
    for index, row in exp_df.iterrows():
        exp_id = format(row['experiment ID'],'02d')
        user_id = format(row['user ID'],'02d')
        start = row['start']
        end = row['end']
        
        acc_exp_path = exp_path + 'acc_exp' + exp_id + '_user' + user_id + '.txt'
        gyro_exp_path = exp_path + 'gyro_exp' + exp_id + '_user' + user_id + '.txt'
        
        acc_data = pd.read_csv(acc_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        gyro_data = pd.read_csv(gyro_exp_path, delimiter = ' ', header = None, names = ['x','y','z'])
        
        acc_data_list_x.append(acc_data['x'][start:end].values)
        acc_data_list_y.append(acc_data['y'][start:end].values)
        acc_data_list_z.append(acc_data['z'][start:end].values)
        
        gyro_data_list_x.append(gyro_data['x'][start:end].values)
        gyro_data_list_y.append(gyro_data['y'][start:end].values)
        gyro_data_list_z.append(gyro_data['z'][start:end].values)
        
    exp_df['acc x'] = acc_data_list_x
    exp_df['acc y'] = acc_data_list_y
    exp_df['acc z'] = acc_data_list_z
    exp_df['gyro x'] = gyro_data_list_x
    exp_df['gyro y'] = gyro_data_list_y
    exp_df['gyro z'] = gyro_data_list_z
    
    return exp_df


In [ ]:
labels_path = 'HAPT Data Set/RawData/labels.txt'
exp_data = read_experiments(labels_path)
exp_data

In [ ]:
def segment_experiments(exp_df,segment_length,step): 
    #for x,y,z (acc) and x,y,z gyro
    NUM_FEATURES = 3
    features = ['acc x','acc y', 'acc z', 'gyro x', 'gyro y', 'gyro z']
    
    segments = []
    labels = []
    
    for index, row in exp_df.iterrows():
        for i in range(0 , row['end'] - row['start'], step):
            xa = row['acc x'][i : i + segment_length]
            ya = row['acc y'][i : i + segment_length]
            za = row['acc z'][i : i + segment_length]
#             xg = row['gyro x'][i : i + segment_length]
#             yg = row['gyro y'][i : i + segment_length]
#             zg = row['gyro z'][i : i + segment_length]
            
            if len(xa) == segment_length:
                segments.append([xa, ya, za])
                labels.append(row['activity ID'])

    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, segment_length, NUM_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels

In [ ]:
SEGMENT_LENGTH = 100
STEP = 100
activity_labels_path = 'HAPT Data Set/activity_labels.txt' 
LABELS = pd.read_csv(activity_labels_path, delimiter = ' ', header = None)[1].tolist()

segments,labels = segment_experiments(exp_data,SEGMENT_LENGTH,STEP)

divider = int(len(segments) * .7)

x_train = segments[:divider]
y_train = labels[:divider]

x_test = segments[divider:]
y_test = labels[divider:]

num_time_periods, dims = x_train.shape[1], x_train.shape[2]
num_classes = len(LABELS)

input_shape = (num_time_periods * dims)
x_train = x_train.reshape(x_train.shape[0],input_shape)

y_train = y_train - 1

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')


y_train = utils.to_categorical(y_train, num_classes)

# test data

x_test = x_test.reshape(x_test.shape[0],input_shape)

y_test = y_test - 1

x_test = x_test.astype('float32')
y_test = y_test.astype('float32')

y_test = utils.to_categorical(y_test, num_classes)


## CNN MODEL

In [14]:
cnn_model = tf.keras.Sequential()
cnn_model.add( Reshape( (SEGMENT_LENGTH, dims), input_shape=(input_shape,), name='cnn_input' ) )
cnn_model.add( Conv1D(100, 10, activation='relu', input_shape = (SEGMENT_LENGTH,dims)) )
cnn_model.add( Conv1D(100, 10, activation='relu') )
cnn_model.add( Conv1D(100, 10, activation='relu') )
cnn_model.add( MaxPooling1D(3) )
cnn_model.add( Conv1D(160, 10, activation='relu') )
cnn_model.add( Conv1D(160, 10, activation='relu') )
cnn_model.add( GlobalAveragePooling1D() )
cnn_model.add( Dropout(0.5) )
cnn_model.add( Dense(num_classes, activation = tf.nn.softmax ,name='cnn_output') )
print(cnn_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Reshape)          (None, 100, 3)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 91, 100)           3100      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 82, 100)           100100    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 73, 100)           100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 100)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 15, 160)           160160    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 160)           

In [15]:
cnn_model.save('cnn_acconly_untrained.h5')

In [8]:
def buildLstmLayer(inputs, num_layers, num_units):
    """Build the lstm layer.

    Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
    """
    lstm_cells = []
    for i in range(num_layers):
        lstm_cells.append(
            tf.lite.experimental.nn.TFLiteLSTMCell(
                num_units, forget_bias=0, name='rnn{}'.format(i)))
    lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
    # Assume the input is sized as [batch, time, input_size], then we're going
    # to transpose to be time-majored.
    transposed_inputs = tf.transpose(inputs, perm=[1, 0, 2])
    outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
        lstm_layers,
        transposed_inputs,
        dtype='float32')
    unstacked_outputs = tf.unstack(outputs, axis=0)
    return unstacked_outputs[-1]

## RNN Model

In [9]:
tf.reset_default_graph()
rnn_model = tf.keras.Sequential()
rnn_model.add( Reshape((SEGMENT_LENGTH, dims), input_shape = (input_shape,)) )
rnn_model.add( Input(shape=(SEGMENT_LENGTH, dims) ) )
rnn_model.add( Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 100}))
rnn_model.add( Dropout(0.5) )
rnn_model.add( Dense(100, activation = 'relu'))
rnn_model.add( Dense(num_classes, activation = 'softmax') )
print(rnn_model.summary())

W0520 14:03:14.162262 21436 deprecation.py:506] From c:\users\devmi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 100, 3)            0         
_________________________________________________________________
input_1 (InputLayer)         multiple                  0         
_________________________________________________________________
lambda (Lambda)              (None, 100)               122000    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                1212      
Total params: 133,312
Trainable params: 133,312
Non-trainable params: 0
__________________________________________________

In [10]:
rnn_model.save("rnn_acconly_untrained.h5")

## CNN-RNN Hybrid

In [11]:
SUB_STEPS = 5
SUB_LENGTH = 20

tf.reset_default_graph()

cnn_rnn_model = tf.keras.Sequential()
# cnn_rnn_model.add( Reshape((SEGMENT_LENGTH, dims), input_shape = (input_shape,)) )
cnn_rnn_model.add( Reshape((SUB_STEPS,SUB_LENGTH, dims), input_shape = (input_shape,)) )
cnn_rnn_model.add( TimeDistributed(Conv1D(filters = 100, kernel_size = 3,  activation='relu'), input_shape = (None, SUB_LENGTH, dims)))
cnn_rnn_model.add( TimeDistributed(Conv1D(filters = 100, kernel_size = 3,  activation='relu')) )
cnn_rnn_model.add( TimeDistributed(Dropout(0.5)) )
cnn_rnn_model.add( TimeDistributed(MaxPooling1D(pool_size=2)) )
cnn_rnn_model.add( (TimeDistributed(Flatten())) )
cnn_rnn_model.add( Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 100}) )
cnn_rnn_model.add( Dropout(0.5) )
cnn_rnn_model.add( Dense(100, activation = 'relu'))
cnn_rnn_model.add( Dense(num_classes, activation = 'softmax') )
print(cnn_rnn_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 5, 20, 3)          0         
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 18, 100)        1000      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 16, 100)        30100     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 16, 100)        0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 8, 100)         0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 800)            0         
_________________________________________________________________
lambda (Lambda)              (None, 100)               4

In [12]:
cnn_rnn_model.save('cnn_rnn_acconly_untrained.h5')

In [18]:
class EpochEndCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        decay = self.model.optimizer.decay
        iterations = self.model.optimizer.iterations
        lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        print(K.eval(lr_with_decay))

def train(model):
    # The EarlyStopping callback monitors training accuracy:
    # if it fails to improve for two consecutive epochs,
    # training stops early
    callbacks_list = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath='cnn_best_model.{epoch:02d}-{val_loss:.2f}.h5',
            monitor='val_loss', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor='acc', patience=50),
        tf.keras.callbacks.TensorBoard(log_dir='tfb_logs', histogram_freq=0,
              write_graph=True, write_images=True),
#         EpochEndCallback
    ]

    model.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

    BATCH_SIZE = 400
    EPOCHS = 100

    # Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
    history = model.fit(x_train,
                          y_train,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          callbacks=callbacks_list,
                          validation_split=0.2,
                          verbose=1)

In [39]:
train(cnn_model)

Train on 4228 samples, validate on 1057 samples
Epoch 1/50
4228/4228 [==============================] - 9s 2ms/sample - loss: 1.6278 - acc: 0.4756 - val_loss: 1.0839 - val_acc: 0.6452
Epoch 2/50
4228/4228 [==============================] - 5s 1ms/sample - loss: 1.0251 - acc: 0.6518 - val_loss: 0.7026 - val_acc: 0.7644
Epoch 3/50
4228/4228 [==============================] - 5s 1ms/sample - loss: 0.7587 - acc: 0.7327 - val_loss: 0.5561 - val_acc: 0.8079
Epoch 4/50
4228/4228 [==============================] - 5s 1ms/sample - loss: 0.6680 - acc: 0.7588 - val_loss: 0.5653 - val_acc: 0.8070
Epoch 5/50
4228/4228 [==============================] - 5s 1ms/sample - loss: 0.6013 - acc: 0.7826 - val_loss: 0.4971 - val_acc: 0.8297
Epoch 6/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.5668 - acc: 0.7938 - val_loss: 0.5503 - val_acc: 0.8297
Epoch 7/50
4228/4228 [==============================] - 6s 1ms/sample - loss: 0.5000 - acc: 0.8172 - val_loss: 0.4269 - val_acc: 0.8836


In [9]:
score = rnn_model.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

NameError: name 'rnn_model' is not defined

## Load, Train and Test Models

In [19]:
cnn_model = tf.keras.models.load_model('cnn_acconly_untrained.h5')
train(cnn_model)

W0520 14:05:31.397998 21436 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


Train on 4228 samples, validate on 1057 samples


InvalidArgumentError: Node 'training/Adam/gradients/dropout_2_2/cond_grad/If': Connecting to invalid output 3 of source node dropout_2_2/cond which has 1 outputs

### test model


In [9]:
score = cnn_model.evaluate(x_test, y_test, verbose=1)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

2266/2266 [==============================] - 1s 311us/sample - loss: 0.5609 - acc: 0.8694

Accuracy on test data: 0.87

Loss on test data: 0.56


## test tflite

In [26]:
path = 'CNN_2.tflite'
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.get_input_details()

x_test = np.array([np.genfromtxt('test.txt',delimiter = ',')]).astype('float32')



try:
    interpreter.allocate_tensors()
except ValueError:
    assert False

MINI_BATCH_SIZE = 1
correct_case = 0
for i in range(len(x_test)):
    input_index = (interpreter.get_input_details()[0]['index'])
    interpreter.set_tensor(input_index, x_test[i * MINI_BATCH_SIZE: (i + 1) * MINI_BATCH_SIZE])
    interpreter.invoke()
    output_index = (interpreter.get_output_details()[0]['index'])
    result = interpreter.get_tensor(output_index)
    # Reset all variables so it will not pollute other inferences.
    interpreter.reset_all_variables()
    # Evaluate.
    print(result)
    prediction = np.argmax(result)
    if prediction == np.argmax(y_test[i]):
        correct_case += 1

print('TensorFlow Lite Evaluation result is {}'.format(correct_case * 1.0 / len(x_test)))

[[3.4892527e-04 1.9143837e-02 1.9276749e-01 7.4593164e-03 4.5205603e-04
  3.3515282e-02 1.3556823e-01 2.1252704e-03 5.2138116e-02 1.1267266e-02
  5.1720178e-01 2.8012501e-02]]
TensorFlow Lite Evaluation result is 0.0
